# Iterative Quantum Phase Estimation (IQPE)

Present notebook reviews an **amplitude amplification** algorithm that does not rely $\mathcal{QFT}^{-1}$: the **Iterative Quantum Phase Estimation** algorithm (**IQPE**).

**Iterative Phase Estimation** is a more general algorithm than the **Amplitude Estimation** algorithms (like the **maximum likelihood** one). In general the **IQPE** algorithm can be used for estimating phase autovalues of unitary operators. 


Present notebook and module are based on the following references:

* *Dobšíček, Miroslav and Johansson, Göran and Shumeiko, Vitaly and Wendin, Göran*. Arbitrary accuracy iterative quantum phase estimation algorithm using a single ancillary qubit: A two-qubit benchmark. Physical Review A 3(76), 2007. https://arxiv.org/abs/quant-ph/0610214

* *Griffiths, Robert B. and Niu, Chi-Sheng*. Semiclassical Fourier Transform for Quantum Computation. Physical Review Letters, 17 (76), 1996. https://arxiv.org/abs/quant-ph/9511007

* *A. Y. Kitaev*. Quantum measurements and the abelian stabilizer problem. Electronic Colloquium on Computational Complexity, 3(3):1–22, 1996. https://arxiv.org/abs/quant-ph/9511026

* *Monz, Thomas and Nigg, Daniel and Martinez, Esteban A. and Brandl, Matthias F. and Schindler, Philipp and Rines, Richard and Wang, Shannon X. and Chuang, Isaac L. and Blatt, Rainer*. Realization of a scalable Shor algorithm. Science 6277 (351). 2016. https://arxiv.org/abs/1507.08852

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
import sys
sys.path.append("../")

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from libraries.utils.qlm_solver import get_qpu
QLMaaS = True
linalg_qpu = get_qpu(QLMaaS)

## 1. Oracle generation

Before doing any amplitude estimation we want to load some data into the quantum circuit, as this step is only auxiliary to see how the algorithm works, we are just going to load a discrete probability distribution. In this case we will have a circuit with $n=3$ qubits which makes a total of $N = 2^n = 8$ states. The discrete probability distribution that we are going to load is:
$$p_d = \dfrac{(0,1,2,3,4,5,6,7)}{0+1+2+3+4+5+6+7+8}.$$

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)

Note that this probability distribution is properly normalised. For loading this probability into the quantum circuit we will use the function *load_probability*. The state that we are going to get is:
    $$|\Psi\rangle = \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].$$

In [ ]:
from libraries.DL.data_loading import load_probability, load_array

In [ ]:
oracle = load_probability(probability)

In [ ]:
%qatdisplay oracle

For more information about loading data into the quantum circuit see the notebook *data_loading_use*.

## 2. Grover-like operator from Oracle

The problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $\sqrt{a}$. We can define an associated angle to $\sqrt{a}$ as $\sin^2{\theta} = a$, and the problem is thus rewritten as:
$$\mathcal{O}|0\rangle = |\Psi \rangle = \sin(\theta)|\Psi_0\rangle +\cos(\theta)|\Psi_1\rangle,$$

The foundations of any amplitude estimation algorithm is the grover operator $\mathcal{Q}$. We recall that the grover operator has the following effect over our state $|\Psi\rangle$:
$$\mathcal{Q}^{m}|\Psi\rangle = |\Psi \rangle = \sin\left((2m_k+1)\theta\right)|\Psi_0\rangle +\cos\left((2m_k+1)\theta\right)|\Psi_1\rangle,$$


The eigenvalues of grover operator $\mathcal{Q}$ are the phases: $e^{\pm i \theta}$. 
The $\theta$ is the same for the eigenvalues ($e^{\pm i \theta}$) and for the amplitude amplification 
$\sin\left((2m_k+1)\theta\right)$. 

With the *IPE* algorithm we can estimate this $\theta$ but the grover operator $\mathcal{Q}$ is mandatory.

Using example from section 1 we are going to define the following amplitude estimation problem:
$$
    \begin{array}{l}
    &\mathcal{O}\longrightarrow \mathcal{P}.\\
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sin(\theta)|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \cos(\theta)|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

In this case we are going to use **grover** function from **amplitude_amplification** module for creating the correspondiente *Grover* operator for the beforementioned oracle. For more information about this function and module see the notebook **02_AmplitudeAmplification_Operators**. For the section 1 case the mandatory inputs of the grover function wil be:

* oracle operator
* target state: $|1\rangle$ wich binnary representation is: $001$
* Qbits where we are going to act:  $[0,1,2]$, the whole register.





In [ ]:
from libraries.AA.amplitude_amplification import grover

In [ ]:
target = [0, 0, 1]
index = range(oracle.arity)
q_gate = grover(oracle, target, index)

In [ ]:
%qatdisplay q_gate --depth 2

So now we have the oracle and the correspondient grover operator.

## 2. Class IQPE: algorithm step by step 

We have implemented and python class called **IQPE** (in the script **iterative_quantum_pe.py**) that allows us implement the **IQPE** algorithm. In this section we are going to describe the class step by step and explain the basics of the **IQPE** algorithm


### 2.1 Calling the **IQPE** class

In [ ]:
#Load Class
from libraries.PE.iterative_quantum_pe import IterativeQuantumPE

In order to instantiate the class we need to provide a pyhton dictionary. Most important keys are:

* initial_state : QLM Program with an initial state $|\Psi\rangle$ was loaded. 
* unitary_operator :  QLM gate or routine with an Unitaryt operator (in this case the Grover-like operator $\mathcal{Q}$) ready for be applied to initial state $|\Psi\rangle$.


Other important keys are:

* cbits_number : int with the number of classical bits needed for for phase estimation
* qpu : QLM solver. If not provided class try to creates a PyLinalg solver. It is recomended give this key to the class.
* shots : int number of shots for quantum job. If 0 exact probabilities will be computed


In [ ]:
n_cbits = 7
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': oracle,
    'unitary_operator': q_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    #'easy': True,
    #'easy': False    
}
IQPE = IterativeQuantumPE(**iqpe_dict)

When the class is instantiated the properties *initial_state* and *q_gate* are overwritten with the given keys **initial_state** and **unitary_operator** respectively

In [ ]:
c = IQPE.initial_state
%qatdisplay c

In [ ]:
a = IQPE.q_gate
%qatdisplay a --depth 2

* Alberto: Wall time: 9.42 s
* Gonzalo: Wall time: 663 ms

### 2.2 IPE Algorithm step by step

Now we are going to review step by step the **IPE** algorithm using different programed methods of the **IterativeQuantumPE** class

### 2.2.1. Initialize the quantum program.

First thing is calling the method **init_iqpe**. Following actions are done by this method:
1. Creation of QLM program from *initial_state* QLM routine (or AbstractGate). The QLM program is stored in *q_prog* property.
2. Allocation of an auxiliar qbit mandatory for the **IPE** algorithm. It is stored in the *q_aux* property.
3. Allocation of the auxiliar classical bits where the estimated phase will be stored. Property: *c_bits*.

In [ ]:
#Initialize the quantum program
IQPE.init_iqpe()

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
#Additionally a auxiliar qbit bits was allocated
circuit = IQPE.q_prog.to_circ(submatrices_only=True)

%qatdisplay circuit --depth 0

### 2.2.2. IPE Algorithm

We are going to decomposed the **IPE** algorithm in 2 parts. A first part where the main variable $l$ will be 0 ($l=0$) and a second recursive part where the variable $l$ will be greater than 0 ($l\gt 0$.

#### First Part ($l=0$)

The first step of the IPE algorithn ($l=0$) has the following parts:

1. Reset the auxiliar qbit
2. Apply a Haddamard gate to the auxiliar qbit
3. Apply a controlled by auxiliar qbit grover-like operator $2^{m-1}$ ($\mathcal{Q} ^{2^{m-1}}$) times with $m$ is the number of classical qbits allocated for estimating $\theta$ 
4. Apply a Haddamard gate to the auxiliar qbit
5. Measuring the auxiliar qbit and store the result into classical bit array in position $c_l$ ($l=0$)

This can be done by calling the *step_iqpe* method with following arguments:

* Quantum Program with initial_state
* Quantum Routine or AbstractGate with Grover-like operator $\mathcal{Q}$
* Auxiliar Qbit
* Auxiliar classical bits
* l=0

This methods return the quantum program with the operations explained in this part.

In [ ]:
q_program = IQPE.q_prog
q_program = IQPE.step_iqpe(q_program, IQPE.q_gate, IQPE.q_aux, IQPE.c_bits, 0)

Following cell show the first  part of the circuit

In [ ]:
c = q_program.to_circ()
%qatdisplay c

#### Second or iterative Part ($l \gt 0$)

This part of the algorithm is recursive. Following steps will be repeated for each value of $l=1,2,..m-1$:

1. Reset the auxiliar qbit
2. Apply a Haddamard gate to the auxiliar qbit
3. Apply a controlled by auxiliar qbit grover-like operator $2^{m-1-l}$ ($\mathcal{Q}^{2^{m-1-l}}$) times with $m$ is the number of classical qbits allocated for estimating $\theta$.
4. Apply on the auxiliar qbit a set of controlled rotations by $c_j$ classical bit of angle: $\frac{\pi}{2}\frac{1}{2^{l-j-1}}$ with $j=0,1,..l-1$. 
4. Apply a Haddamard gate to the auxiliar qbit
5. Measuring the auxiliar qbit and store the result into classical bit array in position $c_l$

So for a $l$ step de controlled by classical bits rotation will depend on the measurements don on the before $l$ steps.

In the following cell we explain how to create the algorithm for the $l=1$ part:

In [ ]:
#here we do the step l=1
l=1
q_program = IQPE.step_iqpe(q_program, IQPE.q_gate, IQPE.q_aux, IQPE.c_bits, l)

Now we can plot the circuit we have unitl the moment

In [ ]:
#Circuit for l=0 and l=1
c = q_program.to_circ()
%qatdisplay c

####  Complete algorithm

For a complete **IPE** algorithm following steps shold be done:

1. Create the First part of the algorithm $l=0$.
2. Iterate the second part of the algorihtm from  $l=1$ to $l=m-1$.

The measured classical bits is used for estimating the phase autovalues of the unitary operator

Following cell create the complete program for **IPE** algorithm

In [ ]:
#Initialize the quantum program
IQPE.init_iqpe()
q_program = IQPE.q_prog
for l in range(len(IQPE.c_bits)):
    q_program = IQPE.step_iqpe(q_program, IQPE.q_gate, IQPE.q_aux, IQPE.c_bits, l)

So for the desired number of classical bits $m$ the complete circuit for *IPE* algorithm will be:

In [ ]:
c = q_program.to_circ()
%qatdisplay c

### 2.2.3. IPE Algorithm execution

Once the QLM program is constructed the alogrithm should be executed. For this the *run* method from the class allow to execute it. Following arguments should  be provided:

* q_prog : with the complete *IPE* algorithm
* q_aux : the auxiliar qbit 
* shots 
* linalg_qpu: QLM solver

This method creates the circuit, the asociated job and execute it. The raw results of the simulation are returned


In [ ]:
raw_results = IQPE.run(q_program, IQPE.q_aux, 100, linalg_qpu=linalg_qpu)

In [ ]:
raw_results

### 2.2.4. IPE: getting classical bits measurements

As explained the phase will be estimated by getting the measurements of the classical bits. In the class this is done by the **meas_classical_bits** method. The input of this method is the *raw_results* from the *run* method. And the output will be a pandas DataFrame with the measurement of the classical bits with the following columns:

* **BitString**: is the result of the clasical bits measurement in each step of the algorithm
* **BitInt**: integer representation of the **BitString**
* **Phi**: is the estimated obtained phase and it is computed as: $\frac{BitInt}{2^{m}}$ where $m$ is the number of classical bits used for phase estimation

In [ ]:
classical_bits = IQPE.meas_classical_bits(raw_results)

In [ ]:
classical_bits

### 2.2.5. IPE: post proccessing

For getting the propper $\theta$ from classical bit measurements the *post_proccess* method can be used. The input will be the DataFrame with the classical bits and the output a pandas dataframe with the following columns:

* **BitString**: is the result of the clasical bits measurement in each step of the algorithm
* **BitInt**: integer representation of the **BitString**
* **Phi**: is the estimated obtained phase and it is computed as: $\frac{BitInt}{2^{c_b}}$ where $c_b$ is the number of classical bits 
* **Theta Unitary**: is the phase eigenvalue of the Grover-like operator (2*$\pi$*Phi).
* **Theta**: is the rotation angle $\theta$ applied for the Grover-like operator ($\pi$*Phi)
* **theta_90**: is the rotation angle $\theta$ applied for the Grover-like operator ($\pi$*Phi) between $(0, \frac{\pi}{2}$)

In [ ]:
final_results = IQPE.post_proccess(classical_bits)

In [ ]:
final_results

The DataFrame from *post_proccess* gives a result for each execution of the circuit (variable *shots*) with the *sumarize* method we can obtain the freqcuency for a given column of the DataFrame. The input are:

* *final_results*: DataFrame from *post_proccess* method
* columns : list with the columns user want to get the frequency

In [ ]:
#frequencies for the column theta_90
IQPE.sumarize(final_results, ['theta_90'])

In [ ]:
#Freqeuncy for all columns
IQPE.sumarize(final_results, list(final_results.columns))

For the use case we implemented we want to use the *theta_90* for gettign the measured $\theta$. From this values we can compute:

$$\cos^2(\theta)$$

for computing the probability of getting 
$$\sin(\theta)|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle$$

In [ ]:
pdf = IQPE.sumarize(final_results, ['theta_90'])
pdf['P']=np.cos(pdf['theta_90'])**2

In [ ]:
pdf

In [ ]:
print("Classical result: ",probability[1])
print("Quantum result: ",pdf['P'].iloc[0])

## 3. Class IQPE: complete execution

All the *IPE* steps explained in section 2 can be done with the method **iqpe**. When using this *method* following properties are populated:

* *classical_bits*: the DataFrame with the result of the *meas_classical_bits* method.
* *final_results*: the DataFrame with the result of the *post_proccess* method.
* *sumary*: the DataFrame with the result of the *sumarize* method.

In [ ]:
n_cbits = 10
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': oracle,
    'unitary_operator': q_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_ = IterativeQuantumPE(**iqpe_dict)

In [ ]:
iqpe_.iqpe()

In [ ]:
iqpe_.final_results

In [ ]:
iqpe_.sumary

In [ ]:
iqpe_.sumary['P']=np.cos(iqpe_.sumary['theta_90'])**2

In [ ]:
iqpe_.sumary

With this measurements histograms can be plotted in order to have an insight of the frequencies for the looked measurements

In [ ]:
print("Classical result: ",probability[1])
print("Quantum result: ",iqpe_.sumary['P'].iloc[0])

## 4. Qiskit Test

We are going to use the **IQPE** example from Qiskit textbook for showing this. Following links have the Qiskit examples:

https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb


In [ ]:
#Number Of Qbits
n_qbits = 1
#Number Of Classical Bits
n_cbits = 2

In the Qiskit example they try to estimate the phase for 

![title](Qiskit_IQPE.png)

In [ ]:
import qat.lang.AQASM as qlm

In [ ]:
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])
grover = qlm.PH(np.pi/2.0)    

In [ ]:
%qatdisplay initial_state
%qatdisplay grover

In [ ]:
from libraries.PE.iterative_quantum_pe import IterativeQuantumPE

In [ ]:
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': grover,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    'shots': 1000,
}
IQPE = IterativeQuantumPE(**iqpe_dict)
IQPE.iqpe()

In [ ]:
IQPE.final_results

In [ ]:
plt.hist(IQPE.final_results['Phi'])

In [ ]:
IQPE.final_results['Phi'].describe()

In [ ]:
IQPE.sumarize(IQPE.final_results, ['Phi'])

As can be seen in 

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb 

solution in qiskit is just 0.25 for the before configuration